In [16]:
import os
import sys
sys.path.append('../src')

from openai import OpenAI
from dotenv import load_dotenv
from agents import BaseAgent, Agent, MultiAgent
from pydantic import BaseModel

In [17]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)


In [18]:
# Top level agent
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if you don't know how to answer the user's question or do not has access to the necessary information.")

In [19]:
class AgentDesc(BaseModel):
    name: str
    instructions: str
    handoff_description: str
    
def create_new_agent(input: str)->Agent:
    """ 
    If there is no other agent or tool can handle the input, call this function to create a new agent.
    """

    model = "gpt-4o-mini"

    completion = client.beta.chat.completions.parse(
    model=model,
    messages=[
        {"role": "system", "content": """ 
         Create a new agent. The agent should be able to handle user request. return the agent description.

        ## name example
        
         science agent (only english characters and spaces are allowed)
         
        ## instructions example
         
        You are a Science Agent As a science agent, you have a strong understanding of physics, chemistry, and biology. You can answer a variety of questions in these fields. Your aim is to make science interesting and easy to understand for everyone. If a question is outside your expertise, you can refer the user to another agent.
         
        ## handoff_description example
        
        Call this agent if you have a question related to science, physics, chemistry, or biology. This agent can provide detailed explanations, clarify complex concepts, and offer insights into the natural world.
         
         """},
        {"role": "user", "content": input},
    ],
    response_format=AgentDesc,
)

    event = completion.choices[0].message.parsed

    new_agent = Agent(base_agent = BaseAgent(client),name=event.name, instructions=event.instructions, handoff_description=event.handoff_description)

    general_agent.add_handoff(new_agent)
    new_agent.add_handoff(general_agent)

    return "#CreateNewAgent#"
    


In [20]:
general_agent.add_tools([create_new_agent])

In [21]:
multi_agent = MultiAgent(start_agent=general_agent)

In [22]:
multi_agent.chat([{ "role": "user", "content": "how to ride a bike?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_lK9JaND8q9gOVQivbaPY9LyO',
    'function': {'arguments': '{"input":"how to ride a bike"}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_lK9JaND8q9gOVQivbaPY9LyO',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'content': 'I can help you with that! Here are some basic steps to ride a bike:\n\n1. **Get the Right Bike**: Make sure the bike is the right size for you. When seated, you should be able to touch the ground with your toes.\n\n2. **Wear a Helmet**: Safety first! Always wear a helmet to protect your head in case of a fall.\n\n3. **Find a Safe Location**: Look for a flat, open area with minimal traffic, like a park or an empty parking lot.\n\n4. **Learn to Balance**: Start by walking alongside the bike. Sit on the saddle and walk the bike while balancing.\n\n5. **Use t

In [24]:
multi_agent.chat([{ "role": "user", "content": "what kind of bike should I buy if i want to ride in the mountain?"}],show_details=True)

[{'role': 'assistant',
  'content': "If you're looking to ride in the mountains, a mountain bike is your best option. Here are a few key factors to consider when choosing a mountain bike:\n\n1. **Type of Mountain Bike**:\n   - **Hardtail**: These bikes have a front suspension but no rear suspension, making them lighter and more efficient for climbing. They are good for cross-country trails.\n   - **Full-Suspension**: These bikes have both front and rear suspension, providing better control and comfort on rough terrain. They are ideal for technical trails and descents.\n\n2. **Wheel Size**:\n   - **26-inch wheels**: Traditional size, good for maneuverability but less common nowadays.\n   - **27.5-inch wheels**: A good compromise between speed and stability.\n   - **29-inch wheels**: Better for rolling over obstacles and maintaining speed, great for cross-country terrain.\n\n3. **Frame Material**:\n   - **Aluminum**: Lightweight and reasonably priced, ideal for beginners.\n   - **Carbon 

In [25]:
multi_agent.chat([{ "role": "user", "content": "can you explain the theory of relativity?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_CAo1vhxrjDgFUg5ucY6Dqsxl',
    'function': {'arguments': '{"message": "can you explain the theory of relativity?"}',
     'name': 'general_agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'Bike_Riding_Agent'},
 {'role': 'tool',
  'handoff': 'Bike_Riding_Agent -> general_agent',
  'content': 'Handing off to general_agent',
  'tool_call_id': 'call_CAo1vhxrjDgFUg5ucY6Dqsxl',
  'message': 'can you explain the theory of relativity?',
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_85Jnn9zXDUjm5PBWWjbkd89T',
    'function': {'arguments': '{"input":"Explain the theory of relativity."}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_85Jnn9zXDUjm5PBWWjbkd89T',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'c

In [26]:
general_agent.handoff_agent_schemas

[{'type': 'function',
  'function': {'name': 'Bike_Riding_Agent',
   'description': "You are a Bike Riding Agent. As a bike riding agent, you have comprehensive knowledge about how to ride various types of bicycles, safety tips, techniques for balancing, pedaling, steering, and overcoming common difficulties new riders face. Your goal is to provide step-by-step guidance and make cycling accessible and enjoyable for everyone. If a user's question is outside your expertise, you can refer them to another agent.",
   'parameters': {'type': 'object',
    'properties': {'message': {'type': 'string',
      'description': 'The message to send to the agent.'}},
    'required': ['message']}}},
 {'type': 'function',
  'function': {'name': 'Science_Agent',
   'description': 'You are a Science Agent. As a science agent, you have a strong understanding of physics, chemistry, and biology. You can answer a variety of questions in these fields. Your aim is to make science interesting and easy to unders

In [27]:
multi_agent.chat([{ "role": "user", "content": "what is the speed of light?"}],show_details=True)

[{'role': 'assistant',
  'content': 'The speed of light in a vacuum is approximately \\(299,792,458\\) meters per second (m/s), which is often rounded to \\(3.00 \\times 10^8\\) m/s for simplicity in calculations. This speed is a fundamental constant of nature and is denoted by the symbol \\(c\\). Light travels at this speed in a vacuum, but it can slow down when passing through different materials, such as water or glass.',
  'sender': 'Science_Agent'}]

In [28]:
multi_agent.chat([{ "role": "user", "content": "what to eat for breakfast?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_L6m1XDquOYSDVP0f5Y79mW9u',
    'function': {'arguments': '{"message": "what to eat for breakfast?"}',
     'name': 'general_agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'Science_Agent'},
 {'role': 'tool',
  'handoff': 'Science_Agent -> general_agent',
  'content': 'Handing off to general_agent',
  'tool_call_id': 'call_L6m1XDquOYSDVP0f5Y79mW9u',
  'message': 'what to eat for breakfast?',
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_7RpxHesJwc83AkbNAWG41gvI',
    'function': {'arguments': '{"input":"I need advice on what to eat for breakfast."}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_7RpxHesJwc83AkbNAWG41gvI',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'content': 'I can help you fin